<a href="https://colab.research.google.com/github/almedida/thesis/blob/main/estimate_m0s_10k_simulation_new_method_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
install.packages('pacman')
library(pacman)

p_load("tidyverse", "tmvtnorm" )

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘zoo’, ‘sandwich’, ‘mvtnorm’, ‘gmm’



tmvtnorm installed



##**Load Dataset**##

In [2]:
#load dataset on data_df
data_df = read.table("ttest_limma_pvalue_10k_sorted.csv", header= TRUE, sep=",")

pval_raw = as.data.frame(as.matrix(data_df))
pvals1 <- (as.matrix(pval_raw[,1]))
pvals2 <- (as.matrix(pval_raw[,2]))


In [ ]:
length(pvals1)
length(pvals2)

#dim(pvals2)
head(pvals1, 5)

[1] 10000

[1] 10000

0.031585206
0.000428599
0.057169023
0.310835844
0.018508069


##**Calculate cut-off points using Histogram-based method**##

calc.cutoff slightly modifies the estimate.m0 function to return the appropriate p-value cutoff where all p-values greater than the cutoff are assumed to come from null cases.

In [3]:
calc.cutoff = function(p, B = 20, max=1){

  m <- length(p)
  m0 <- m
  bin <- c(-0.1, (1:B)/B*max)
  bin.counts=rep(0,B)

  for(i in 1:B){
    bin.counts[i]=sum((p>bin[i])&(p<=bin[i+1]))
  }

  tail.means <- rev(cumsum(rev(bin.counts))/(1:B))
  temp <- bin.counts - tail.means
  index <- min((1:B)[temp <= 0])
  cutoff2 <- (index)/B*max
  if(cutoff2 == 1) {cutoff2 <- 1-1/B}

  return(cutoff2)

}


In [4]:
cutoff_value1 = calc.cutoff(pvals1, B=20, max=1)
cutoff_value2 = calc.cutoff(pvals2, B=20, max=1)

cutoff = cbind(c(cutoff_value1), c(cutoff_value2))

colnames(cutoff) = c("cutoff_value1", "cutoff_value2")

cutoff

cutoff_value1,cutoff_value2
0.3,0.15


**Filter dataset using cutoff points**
* selecting pvalues in the upper-right quadrant using Histogram-Based method
*  defined as lambda1 = cutoffvalue1 and lambda2 = cutoffvalue2 for pvalue 1 and pvalue 2 respectively.
*  lambda1, lambda2 - probability of pvalue 1 and pvalue 2 in the upper-right quadrant respectively 


convert lambda(truncation points) to z values

selecting the lower bounds (minimum z values) of the bivariate z values

##**Estimating m00s**##

*  m0.1 - DE genes in experiment 1
*  m0.2 - DE genes in experiment 2
*  m11 - DE genes in both experiments
*  m00 - EE genes in both experiments

In [16]:
pval_to_zval <- function(p1, p2, B=B, max=1){

  p_vals = pval_raw  %>% filter(pvalue >=cutoff_value1, limma_pvalue>=cutoff_value2)


  cutoff_value1 = calc.cutoff(pvals1, B=20, max=1)
  cutoff_value2 = calc.cutoff(pvals2, B=20, max=1)

  z_val = as.data.frame(qnorm(as.matrix(p_vals), lower.tail = TRUE))
  colnames(z_val) = c("zvals1", "zvals2")

  zvals1 <- (as.data.frame(as.matrix(z_val[,1])))
  zvals2 <- (as.data.frame(as.matrix(z_val[,2])))

  z_val_extremums = as.data.frame(qnorm(as.matrix(cbind(c(cutoff_value1,1),c(cutoff_value2,1))), lower.tail = TRUE))

  min_z1 <- z_val_extremums[1,1]
  min_z2 <- z_val_extremums[1,2]

  return z_val
  return min_z1
  return min_z2
}

ERROR: ignored

In [ ]:
z <- pval_to_zval(pvals1, pvals2, B=20, max=1)
z

In [13]:
estimate.m0s <- function(p1, p2, B=20){
  m <- length(p1)

  ##find lambda cutoffs using histogram-based method
  c1 <- calc.cutoff(p1, B=B, max=1)
  c2 <- calc.cutoff(p2, B=B, max=1)

  #call function pval_to_zval
  pval_to_zval(p1, p2, B=B, max=1)

  ##estimate m0 for experiment 1
  ind1 <- (p1>=c1)
  m0.1 <- sum(ind1)/(1-c1)

  ##estimate m0 for experiment 2  
  ind2 <- (p2>=c2)
  m0.2 <- sum(ind2)/(1-c2)


  ##estimate m00
  ind12 <- ind1 & ind2
  nA <- sum(ind12)
  #pA <- (1-c1)*(1-c2)
  #m00 <- nA/pA
  
  #here, we used converted pvalues to z values to estimnate m00
  # density function for each row of the bivariate z values (x) and 
  # estimated parameters(rho)
  density = function(x, rho)
  {
    sigma = matrix(c(1, rho, rho, 1), 2, 2)
    z = dtmvnorm(x, mean = c(0,0), sigma = sigma, lower = c(min_z1, min_z2))
  }
  
  # log likelihood of the joint densities
  log_likelihood_fn = function(rho){
    
    joint_likelihood = z_val %>% split(.$zvals2) %>% map_dfr(~density(c(.$zvals1,.$zvals2),rho))    
    return(-sum(log(joint_likelihood)))
    
  }
  
  #MLE of the log likelihood function
  optimal_rho = optimize(log_likelihood_fn, lower = -1, upper = 1 )
  optimal_rho = as.data.frame(optimal_rho)
  
  #probability of a random variable greater than cutoff values 
  rho = as.numeric(optimal_rho[1])
  pA = pmvnorm(lower=c(min_z1, min_z2), upper=c(Inf, Inf), mean=c(0,0), sigma = matrix(c(1, rho, rho, 1), 2, 2))
  
  m00 <- nA/pA
  
  ##estimate m11
  m11 <- sum(m - m0.1 - m0.2 + m00)
  
  ret <- list()
  ret$ms <- c(m, m0.1, m0.2, m11, m00)
  names(ret$ms) <- c("m", "m0.1", "m0.2", "m11","m00")
  ret$cutoffs <- c(c1, c2)
  return(ret)
}

In [14]:
estimate.m0s(pvals1, pvals2, B=20)

ERROR: ignored